#  자연어 처리

In [3]:
import nltk
txt = "Hello , it's me. I was wondering if after all these years. You'd like to meet , to go over everything. They say that time's supposed to heal ya."

sentences = nltk.tokenize.sent_tokenize(txt)
sentences

["Hello , it's me.",
 'I was wondering if after all these years.',
 "You'd like to meet , to go over everything.",
 "They say that time's supposed to heal ya."]

### word token과 pos_tag()

In [5]:
tokens = [nltk.tokenize.word_tokenize(sen) for sen in sentences]
tokens

[['Hello', ',', 'it', "'s", 'me', '.'],
 ['I', 'was', 'wondering', 'if', 'after', 'all', 'these', 'years', '.'],
 ['You',
  "'d",
  'like',
  'to',
  'meet',
  ',',
  'to',
  'go',
  'over',
  'everything',
  '.'],
 ['They', 'say', 'that', 'time', "'s", 'supposed', 'to', 'heal', 'ya', '.']]

In [7]:
pas_tagged_tokens = [nltk.pos_tag(t) for t in tokens]
pas_tagged_tokens

[[('Hello', 'NNP'),
  (',', ','),
  ('it', 'PRP'),
  ("'s", 'VBZ'),
  ('me', 'PRP'),
  ('.', '.')],
 [('I', 'PRP'),
  ('was', 'VBD'),
  ('wondering', 'VBG'),
  ('if', 'IN'),
  ('after', 'IN'),
  ('all', 'PDT'),
  ('these', 'DT'),
  ('years', 'NNS'),
  ('.', '.')],
 [('You', 'PRP'),
  ("'d", 'MD'),
  ('like', 'VB'),
  ('to', 'TO'),
  ('meet', 'VB'),
  (',', ','),
  ('to', 'TO'),
  ('go', 'VB'),
  ('over', 'RP'),
  ('everything', 'NN'),
  ('.', '.')],
 [('They', 'PRP'),
  ('say', 'VBP'),
  ('that', 'IN'),
  ('time', 'NN'),
  ("'s", 'POS'),
  ('supposed', 'VBN'),
  ('to', 'TO'),
  ('heal', 'VB'),
  ('ya', 'NN'),
  ('.', '.')]]

In [24]:
from bs4 import BeautifulSoup as bs
from  urllib.request import urlopen
#url = "http://www.gutenberg.org/files/2554/2555.txt"
url = "http://www.gutenberg.org/files/2554/2554-0.txt"
raw = urlopen(url).read()
#a = nltk.clean_html(raw)
a = bs.get_text(str(raw))
print(a)


AttributeError: 'str' object has no attribute '_all_strings'

In [66]:
import pandas as pd

train = pd.read_csv("data/train.tsv", sep="\t", index_col="PhraseId")
print(train.shape)
train.head()
test = pd.read_csv("data/test.tsv", sep="\t", index_col="PhraseId")
print(test.shape)
test.head()

train["Phrase(origin)"] = train["Phrase"].copy()

print(train.shape)
test["Phrase(origin)"] = test["Phrase"].copy()
test[["Phrase", "Phrase(origin)"]].head()
print(test.shape)
def clean_text(phrase):
    phrase = phrase.replace("doesn't ", "does not ")
    phrase = phrase.replace("ca n't ", "can not ")
    phrase = phrase.replace(" n't ", " not ")
    return phrase

train["Phrase"] = train["Phrase"].apply(clean_text)
print(train.shape)
train[["Phrase", "Phrase(origin)"]].head()
#train

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=10000, ngram_range=(1,2))
vectorizer.fit(train["Phrase"])
x_train = vectorizer.transform(train["Phrase"])
columns = vectorizer.get_feature_names()
pd.DataFrame(x_train[:100].toarray(), columns=columns).head()
#print("columns:",columns)

x_test = vectorizer.transform(test["Phrase"])
x_test

y_train = train["Sentiment"]

from sklearn.linear_model import SGDClassifier
model = SGDClassifier(random_state=37)

from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(model, x_train, y_train, cv=5)
y_pred[0:10]

from sklearn.metrics import accuracy_score
score = accuracy_score(y_train, y_pred)
print("Score = {0:.5f}".format(score))

import numpy as np
rs = train.copy()
rs["Sentiment(pred)"] = y_pred
rs["Difference(Phrase)"] = np.abs(y_train - y_pred)

print(rs.shape)
rs.head()

(156060, 3)
(66292, 2)
(156060, 4)
(66292, 3)
(156060, 4)


c:\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Score = 0.57659
(156060, 6)


,SentenceId,Phrase,Sentiment,Phrase(origin),Sentiment(pred),Difference(Phrase)
PhraseId,,,,,,
1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...,1,0
2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...,2,0
3,1,A series,2,A series,2,0
4,1,A,2,A,2,0
5,1,series,2,series,2,0


In [73]:
sentiment = rs.groupby("SentenceId")["Difference(Phrase)"].mean()
print(sentiment.shape)
sentiment.head(10)

(8529,)


SentenceId
1     0.238095
2     0.444444
3     0.314286
4     0.675000
5     0.800000
6     0.562500
7     0.466667
8     0.235294
9     0.583333
10    0.750000
Name: Difference(Phrase), dtype: float64

In [68]:
def find_sentiment(sentence_id):
    return sentiment.loc[sentence_id]

rs["Difference(Sentence)"] = rs["SentenceId"].apply(find_sentiment)
rs

,SentenceId,Phrase,Sentiment,Phrase(origin),Sentiment(pred),Difference(Phrase),Difference(Sentence)
PhraseId,,,,,,,
1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...,1,0,0.238095
2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...,2,0,0.238095
3,1,A series,2,A series,2,0,0.238095
4,1,A,2,A,2,0,0.238095
5,1,series,2,series,2,0,0.238095
6,1,of escapades demonstrating the adage that what...,2,of escapades demonstrating the adage that what...,2,0,0.238095
7,1,of,2,of,2,0,0.238095
8,1,escapades demonstrating the adage that what is...,2,escapades demonstrating the adage that what is...,3,1,0.238095
9,1,escapades,2,escapades,2,0,0.238095


In [70]:
rs = rs.sort_values(by="Difference(Sentence)", ascending=False)
print(rs.shape)
rs.head()



(156060, 7)


,SentenceId,Phrase,Sentiment,Phrase(origin),Sentiment(pred),Difference(Phrase),Difference(Sentence)
PhraseId,,,,,,,
82700,4270,one of the biggest disappointments of the year,1,one of the biggest disappointments of the year,4,3,2.625
82703,4270,the biggest disappointments,1,the biggest disappointments,2,1,2.625
82702,4270,the biggest disappointments of the year,1,the biggest disappointments of the year,2,1,2.625
82701,4270,of the biggest disappointments of the year,0,of the biggest disappointments of the year,2,2,2.625
82699,4270,is one of the biggest disappointments of the year,0,is one of the biggest disappointments of the year,4,4,2.625


In [72]:
rs[0:10000].to_csv("result.csv")

voca = vectorizer.get_feature_names()
voca[0:3]
pd.DataFrame(voca, columns=["word"]).to_csv("voca.csv")

rs[rs["Phrase"].str.contains("can not recomment")]

model.fit(x_train, y_train)
pred = model.predict(x_test)

pred[0:10]
submission = pd.read_csv("data/sampleSubmission.csv", index_col="PhraseId")
submission["Sentiment"] = pred

print(submission.shape)
submission.head()

submission.to_csv("baseline-script.csv")

c:\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


(66292, 1)


kaggle 에서 가입. AIP생성-다운로드. kaggle 설치 (pip install kaggle)
### kaggle competitions submit -c sentiment-analysis-on-movie-reviews -f baseline-script.csv -m "My first submission(excercise)"

## 어근추출 

In [83]:
import nltk.stem
s = nltk.stem.SnowballStemmer('english')
print ( s.stem("graphics") )
print(s.stem("image"))
print(s.stem("beautiful"))
print(s.stem("buying"))



graphic
imag
beauti
buy


In [85]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer() #부모 클래스를 instance화 해서 build_analyzer를 가져옴
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
    
    

### 통계

### p.112 Stride trick을 이용한 효율적인 rolling average 알고리즘 구현

In [92]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

n = 5; k = 2
a = np.linspace(1,n,n); aid = id (a)
as_strided(a, (k,n), (8,8))
as_strided(a, (k, n-k+1), (8,8))
def shift1(x, k):
    return np.vstack([x[i:n-k+i+1] for i in range(k)])

def shift2(x, k):
    return as_strided(x, (k, n-k+1), (x.itemsize,)*2)

b= shift1(a, k); 
b, id(b) == aid

(array([[1., 2., 3., 4.],
        [2., 3., 4., 5.]]), False)

In [94]:
c= shift2(a, k); 
c, id(c) == aid

(array([[ 1.00000000e+000,  2.00000000e+000,  3.00000000e+000,
          4.00000000e+000,  5.00000000e+000, -1.30063377e-231,
          1.90554803e-315,  1.89816555e-315,  2.72600127e-317,
          4.94065646e-324,  3.05856903e-315, -1.32076236e-231,
          2.42098585e-315,  4.12040555e-315,  5.11135930e-315,
          4.38575916e-315,  1.27640456e-152, -3.88044747e-270,
          6.36598737e-314,  6.36598737e-314,  2.36397787e-315,
          1.90554258e-315,  1.90554116e-315, -1.32139138e-231,
          1.89816555e-315,  1.90554875e-315,  2.50182985e-317,
          4.94065646e-324,  2.38102962e-315, -1.32327843e-231,
          0.00000000e+000,  4.94065646e-324,  0.00000000e+000,
          4.94065646e-324,  7.47022606e-114, -1.32485098e-231,
          2.42127913e-315,  4.94065646e-324,  2.42126858e-315,
          2.42126854e-315,  1.90553475e-315, -1.31981883e-231,
          2.42127676e-315,  4.94065646e-324,  2.42127885e-315,
          2.42127881e-315,  1.90553120e-315, -5.2932024

In [93]:
n, k = 100, 10
t = np.linspace(0., 1., n)
x = t + .1 * np.random.randn(n)
y = shift2(x,k)
x_avg = y.mean(axis=0)

%timeit shift1(x,k)
%timeit shift2(x,k)

16.2 µs ± 46 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.28 µs ± 67.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
